In [0]:
#Importing required libraries.
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import regexp_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')
nltk.download('stopwords')

import scipy.sparse

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm ,ensemble
import xgboost

STEP-1:Importing the dataset and preprocessing


In [0]:
#LoadingS the dataset
dataset=pd.read_csv('IMDB Dataset.csv')

In [0]:
len(dataset)

In [0]:
#Preprocessing the text data
def preprocessor (dataframe_column):
  #converting the text to lower case.
  dataframe_column=dataframe_column.map(lambda s : s.lower())
  #Tokenizing the text data.
  dataframe_column=dataframe_column.map(lambda s : regexp_tokenize(s,"[a-zA-Z_']+") )
  #Removing the stop words.
  stop_words = set(stopwords.words('english')) 
  dataframe_column=dataframe_column.map(lambda s : [w for w in s if not w in stop_words])
  #Detokenizing the tokens to fprm the text data.
  dataframe_column=dataframe_column.map(lambda s : TreebankWordDetokenizer().detokenize(s))
  return dataframe_column

In [0]:
dataset['review']=preprocessor(dataset['review'])

STEP-2:Feature engineering

In [0]:
#Splitting the dataset into train and test datasets.
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset['review'], dataset['sentiment'], test_size=0.5, random_state=63)

In [0]:
#The features created here are in word level.
#Character level features can also be created,by simply changing the analyzer argument from 'word' to 'char'.
#First feature-count vectors.
#Converting the text data into count vectors using CountVectorizer.
count_vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vectorizer.fit_transform(x_train)

xtrain_count =  count_vectorizer.transform(x_train)
xtest_count =  count_vectorizer.transform(x_test)


In [0]:
#Second feature-Tf-idf vectors
#Converting the text data into tf-idf vectors using TfidfVectorizers
tfidf_vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
tfidf_vectorizer.fit(x_train)

xtrain_tfidf =  tfidf_vectorizer.transform(x_train)
xtest_tfidf =  tfidf_vectorizer.transform(x_test)

In [0]:
#Third feature-Count vectors with n-gram
count_vectorizer_ngram = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3))
count_vectorizer_ngram.fit_transform(x_train)

xtrain_count_ngram =  count_vectorizer_ngram.transform(x_train)
xtest_count_ngram =  count_vectorizer_ngram.transform(x_test)


In [0]:
#Fourth feature-Tf-idf vectors with n-grams
tfidf_vectorizer_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3))
tfidf_vectorizer_ngram.fit(x_train)

xtrain_tfidf_ngram =  tfidf_vectorizer_ngram.transform(x_train)
xtest_tfidf_ngram =  tfidf_vectorizer_ngram.transform(x_test)

In [0]:
#For training the model with two or more features, horizontal stack method is used.
X_train_combined_feature = scipy.sparse.hstack([xtrain_tfidf, xtrain_count])
X_test_combined_feature = scipy.sparse.hstack([xtestd_tfidf, xtest_count])

STEP-3:Buildimg the model and evaluating metrics(F1-score)

In [0]:
#Defining a function which returns the F1-score getting the classifier model, and the features as its arguments
def model_f1_score(classifier, train_vector, train_label, test_vector, test_label):
    classifier.fit(train_vector, train_label)
    predictions = classifier.predict(test_vector)

    encoder = preprocessing.LabelEncoder()
    predictions = encoder.fit_transform(predictions)
    test_label = encoder.fit_transform(test_label)
    f1_score = metrics.f1_score(test_label,predictions)
    print('f1_score = ' + str(f1_score))

STEP-4:Training each models and getting their F1-scores

In [0]:
#It is known that Tf-idf vectors give better results than count vectors and n-gram tf-idf vectors.
#Models are trained using only Tf-idf vectors.
#Model 1-Logistic Regression
model_f1_score(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xtest_tfidf, y_test)

In [0]:
#Model 2-Stochastic Gradient Descent (SGD) Classifier.
model_f1_score(linear_model.SGDClassifier(), xtrain_tfidf, y_train, xtest_tfidf, y_test)

In [0]:
#Model 3-Random Forest Classifier.
model_f1_score(ensemble.RandomForestClassifier(), xtrain_tfidf, y_train, xtest_tfidf, y_test)

In [0]:
#Model 4-XG-Boost Classifier.
model_f1_score(xgboost.XGBClassifier(), xtrain_tfidf, y_train, xtest_tfidf, y_test)

In [0]:
#Model 5-Support Vectror Machine (SVM) Classifier.
model_f1_score(svm.SVC(), xtrain_tfidf, y_train, xtest_tfidf, y_test)